In [1]:
import math
import numpy as np
import openrtdynamics2.lang as dy
import openrtdynamics2.py_execute as dyexe

## Fundamental concepts: delays, initial values, and feedback loops

A simple model is given by

    y = dy.signal()                         # introduce variable y
    x = z + u                               # x[k] = z[k] + u[k]
    z << dy.delay(x, initial_state = 2.0)   # z[k+1] = x[k], z[0] = 2.0

Herein, a signal variable $z$ is introduced that is combined with the system input $u$. The resulting intermediate variable $x$ is then delayed and assigned to $z$ closing the feedback loop. The delay has an initial value $z[0] = 2.0$.

Given $u[k] = 1.0$, for $k>=0$, the output sequence for $y$ is then $\{ 2, 3, 4, 5, ...  \}$.

By combination of these elements (and potentially others), more sophisticated signal processing algorithms can be implemented.


In [2]:
dy.clear()

system = dy.enter_system()

# define system inputs
u               = dy.system_input( dy.DataTypeFloat64(1), name='input1', default_value=1.0,  value_range=[0, 25], title="input #1")


y = dy.signal()                         # introduce variable y
x = y + u                               # x[k] = y[k] + u[k]
y << dy.delay(x, initial_state = 2.0)   # y[k+1] = y[k] + x[k], y[0] = 2.0


# define output(s)
dy.append_primay_ouput(y, 'output')

# generate code for Web Assembly (wasm), requires emcc (emscripten) to build
code_gen_results = dy.generate_code(template=dy.WasmRuntime(), folder="generated/delay", build=False)

compiling system simulation (level 0)... 
determining the computation order...
building execution paths...
All dependencies are resolved.
Generated code will be written to generated/delay .


In [3]:
# list the blocks in the diagram
dy.show_blocks()


-------- list of blocks --------

-----------------------------
Blocks in simulation simulation:
-----------------------------
* '1' (1)
  input signals
    - output (double [1]) type proposal: (double [1])
    - input1 (double [1])
  output signals
    - s2 (double [1]) type proposal: (double [1])
* '2' (2)
  input signals
    - s2 (double [1]) type proposal: (double [1])
  output signals
    - output (double [1]) type proposal: (double [1])

  nested subsystems


In [4]:
compiled_system = dyexe.CompiledCode(code_gen_results)
testsim = dyexe.SystemInstance(compiled_system)

In [5]:
sim_results = dyexe.run_batch_simulation(testsim, input_data={}, N=10,  output_keys=['output'] )
sim_results['output']

array([ 2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.])